In [28]:
import json
import regex as re

In [3]:
results = []
with open('cases.json') as user_file:
    
    for line in user_file:
        results.append(json.loads(line))


In [4]:
pattern = re.compile(r'''
’s|’t|’re|’ve|’m|’ll|’d| ?\p{L}+| ?\p{N}+|\s+”
''')
# def get_words(string):

    

In [5]:
re.findall( pattern, results[0]['Snippet'])

['From',
 ' court',
 ' and',
 ' program',
 ' offices',
 ' in',
 ' the',
 ' Bronx',
 ' Brooklyn',
 ' and',
 ' Manhattan',
 ' CASES',
 ' provides',
 ' community',
 'based',
 ' alternatives',
 ' to',
 ' jail',
 ' and',
 ' prison',
 ' These',
 ' programs',
 ' address']

In [7]:
stop_words = set()
with open('../stop_words.txt', 'r') as stop_words_file:
    for line in stop_words_file:
        stop_words.add(line.strip())

In [8]:
import regex as re
pattern = re.compile(r'''
’s|’t|’re|’ve|’m|’ll|’d| ?\p{L}+| ?\p{N}+”
''')

def extract_words(result_list):
    word_lists = []
    vocab = set()
    for result in result_list:
        title = result.get('Title', '')
        snippet = result.get('Snippet', '')
        title_words = re.findall(pattern, title.lower())
        title_words = [word.strip() for word in title_words if word.strip() not in stop_words]
        snippet_words = re.findall(pattern, snippet.lower())
        snippet_words = [word.strip() for word in snippet_words if word.strip() not in stop_words]
        vocab.update(title_words + snippet_words)
        word_lists.append({'title': title_words, 'snippet': snippet_words})
    return word_lists, vocab

In [9]:
wl, vocab = extract_words(results)

In [10]:
wl[0]

{'title': ['cases'],
 'snippet': ['court',
  'program',
  'offices',
  'bronx',
  'brooklyn',
  'manhattan',
  'cases',
  'provides',
  'community',
  'based',
  'alternatives',
  'jail',
  'prison',
  'programs',
  'address']}

In [11]:
feedback = [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

In [12]:
inverse_list = {word: set() for word in vocab}

for i, document in enumerate(wl):
    for word in document['title'] + document['snippet']:
        inverse_list[word].add(i+1)
        

In [13]:
all_documents = set(range(1, 10+1))

In [15]:
def gini_impurity(word, docs_with_word, docs_without_word, feedback):
  relevant_docs_with_word = 0
  relevant_docs_without_word = 0
  
  # import pdb; pdb.set_trace()
  for doc in docs_with_word:
    relevant_docs_with_word += feedback[doc]
  for doc in docs_without_word:
    relevant_docs_without_word += feedback[doc]

  prob_relevant_with_word = relevant_docs_with_word/len(docs_with_word)
  prob_irrelevant_with_word = 1 - prob_relevant_with_word
  
  prob_relevant_without_word = relevant_docs_without_word/len(docs_without_word)
  prob_irrelevant_without_word = 1 - prob_relevant_without_word
  # import pdb; pdb.set_trace()
  impurity_with_word = 1 - (prob_relevant_with_word**2 + prob_irrelevant_with_word**2)
  impurity_without_word = 1 - (prob_relevant_without_word**2 + prob_irrelevant_without_word**2)

  return impurity_with_word, impurity_without_word

In [16]:
percentage_of_relevant_docs = {}

for word, docs in inverse_list.items():
    number_of_relevant_docs = 0
    for doc in docs:
        number_of_relevant_docs += feedback[doc - 1]
    percentage_of_relevant_docs[word] = number_of_relevant_docs/len(docs)

In [17]:
k = 0.6

words_to_search = [word for word in vocab if percentage_of_relevant_docs[word]>k]

In [24]:
len(words_to_search)

26

In [25]:
ranking = {}

for word in words_to_search:
    gini = gini_impurity(word, inverse_list[word], all_documents - inverse_list[word], feedback = {k+1:f for k, f in enumerate(feedback)})
    w1 = len(inverse_list[word])/len(all_documents)
    w2 = 1.0 - w1
    ranking[word] = ( w1*gini[0] + w2*gini[1])

In [26]:

for doc in wl:
    for word in doc['title']:
        if word in words_to_search:
            ranking[word] = ranking[word]*0.95


In [29]:
res = sorted(words_to_search, key= lambda x: ranking[x])

In [30]:
{word: ranking[word] for word in res}

{'data': 0.0,
 'covid': 0.0,
 'cdc': 0.1688888888888889,
 'coronavirus': 0.1688888888888889,
 'worldometer': 0.1688888888888889,
 'tracker': 0.1688888888888889,
 'statistics': 0.1688888888888889,
 'disease': 0.17777777777777778,
 'graphs': 0.17777777777777778,
 'notifiable': 0.17777777777777778,
 'reporting': 0.17777777777777778,
 'number': 0.17777777777777778,
 'cumulative': 0.17777777777777778,
 'reported': 0.17777777777777778,
 'historical': 0.17777777777777778,
 'tracking': 0.17777777777777778,
 'nationally': 0.17777777777777778,
 'deaths': 0.17777777777777778,
 'updated': 0.17777777777777778,
 'pregnancy': 0.17777777777777778,
 'recovered': 0.17777777777777778,
 'routine': 0.17777777777777778,
 'weekly': 0.17777777777777778,
 'supported': 0.17777777777777778,
 'daily': 0.17777777777777778,
 'charts': 0.17777777777777778}

In [31]:
res

['data',
 'covid',
 'cdc',
 'coronavirus',
 'worldometer',
 'tracker',
 'statistics',
 'disease',
 'graphs',
 'notifiable',
 'reporting',
 'number',
 'cumulative',
 'reported',
 'historical',
 'tracking',
 'nationally',
 'deaths',
 'updated',
 'pregnancy',
 'recovered',
 'routine',
 'weekly',
 'supported',
 'daily',
 'charts']